<a href="https://colab.research.google.com/github/danielfang97/CellImageAnalysis_Odelberg/blob/main/NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_json('/content/drive/MyDrive/Subtask_1_train.json')

print(df)

      conversation_ID                                       conversation  \
0                   1  [{'utterance_ID': 1, 'text': 'Alright , so I a...   
1                   2  [{'utterance_ID': 1, 'text': 'I do not want to...   
2                   3  [{'utterance_ID': 1, 'text': 'Oh my God !', 's...   
3                   4  [{'utterance_ID': 1, 'text': 'Barry , I am sor...   
4                   5  [{'utterance_ID': 1, 'text': 'Oh , look , wish...   
...               ...                                                ...   
1369             1346  [{'utterance_ID': 1, 'text': 'Hey Ross , this ...   
1370             1357  [{'utterance_ID': 1, 'text': 'Yeah , but what ...   
1371             1360  [{'utterance_ID': 1, 'text': 'Wow ! Fortunatel...   
1372             1371  [{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...   
1373             1374  [{'utterance_ID': 1, 'text': 'I never sucked ,...   

                                    emotion-cause_pairs  
0     [[3_surprise, 1_I reali

Daniel's code

Recurrent Neural Network (RNN) using Long Short-Term Memory (LSTM)

Step 1: Turn the data into a readable format.

In [6]:
texts = []
emotions = []
speakers = []

for conversation in df['conversation']:
    utterance_texts = [utterance['text'] for utterance in conversation]
    utterance_emotions = [utterance['emotion'] for utterance in conversation]  # Assuming each utterance has an 'emotion' key
    utterance_speakers = [utterance['speaker'] for utterance in conversation]
    texts.append(utterance_texts)
    emotions.append(utterance_emotions)
    speakers.append(utterance_speakers)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import itertools

# Assuming 'texts', 'emotions', and 'speakers' are nested lists

# Flatten the nested lists for emotion and speaker labels
flat_emotions = list(itertools.chain(*emotions))  # Flatten the nested emotion list
flat_speakers = list(itertools.chain(*speakers))  # Flatten the nested speakers list

# Tokenization and Padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts([text for sublist in texts for text in sublist])  # Flatten and fit
sequences = [tokenizer.texts_to_sequences(sublist) for sublist in texts]
padded_sequences = [pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type) for seq in sequences]

# Encoding labels
label_encoder = LabelEncoder()
emotion_labels = label_encoder.fit_transform(flat_emotions)
speaker_labels = label_encoder.fit_transform(flat_speakers)

# Preparing the data for the model
# Your input (X) is 'padded_sequences'
# Your emotion labels (y_emotion) are 'emotion_labels'
# Your speaker labels (y_speaker) are 'speaker_labels'

In [ ]:
# Hyperparameters
vocab_size = 10000   # Replace with your vocabulary size
embedding_dim = 256  # Dimension of embedding layer
lstm_units = 128     # Number of units in LSTM layer
max_length = 50      # Maximum length of input sequences
padding_type = 'post'
trunc_type = 'post'

# Pad sequences for uniformity
X_padded = pad_sequences(X, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Building the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(lstm_units)),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')  # Change the number of units & activation based on your specific task
])

# Compile the model
model.compile(loss='binary_crossentropy',  # Change the loss function based on your specific task
              optimizer='adam',
              metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
model.fit(X_padded, y, epochs=10, validation_split=0.2)  # Adjust epochs, validation split as needed
